# Putting It All Together

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
car_sales = pd.read_csv("car-sales-extended-missing-data.csv")
car_sales.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


In [23]:
car_sales.dtypes

Make              object
Colour            object
Odometer (KM)    float64
Doors            float64
Price            float64
dtype: object

In [24]:
car_sales.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV

np.random.seed(22)

# indexes_to_drop = car_sales[car_sales["Price"].isna() == True].index
# car_sales.drop(index=indexes_to_drop, inplace=True)
car_sales.dropna(subset=["Price", "Doors"], inplace=True)


categorical_features = ["Make", "Colour"]
categorical_transformers = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]
)

odometer_feature = ["Odometer (KM)"]
numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformers, categorical_features),
        ("num", numerical_transformer, odometer_feature)
    ]
)


model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", RandomForestRegressor())
    ]
)


X = car_sales.drop("Price", axis=1)
Y = car_sales["Price"]

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(X, Y, test_size=0.2)


model.fit(X_TRAIN, Y_TRAIN)
model.score(X_TEST, Y_TEST)

0.062327565627240844

In [28]:
# Look how to use Pipeline with GridSearchCV and RandomSearchCV